In [334]:
import csv
import numpy

def matrix_read_csv(path, delimiter=";", mtype="float"):
    reader = csv.reader(open(path, "r"), delimiter = delimiter)
    x = list(reader)
    return numpy.array(x).astype(mtype)

def matrix_print(Title, M):
    print(Title)
    for row in M:
        print([round(x,4)+0 for x in row])

def matrix_print_two(Action, Title1, M1, Title2, M2):
    print(Action)
    print(Title1, '\t'*int(len(M1)/2)+"\t"*len(M1), Title2)
    for i in range(len(M1)):
        row1 = ['{0:+7.3f}'.format(x) for x in M1[i]]
        row2 = ['{0:+7.3f}'.format(x) for x in M2[i]]
        print(row1,'\t', row2)

def matrix_zeros(rows, cols):
    A = []
    for i in range(rows):
        A.append([])
        for j in range(cols):
            A[-1].append(0.0)

    return A

def matrix_copy(matrix):
    rows = len(matrix)
    cols = len(matrix[0])

    matrixcp = matrix_zeros(rows, cols)

    for i in range(rows):
        for j in range(cols):
            matrixcp[i][j] = matrix[i][j]

    return matrixcp

def matrix_array_to_matrix(array):
    result = matrix_zeros(1, len(array))
    for col in range(len(array)):
        result[0][col] = array[col]
    
    return result

def matrix_escalar_matrix_multiplication(matrix_a, matrix_b):
    result = matrix_copy(matrix_a)
    for row in range(len(matrix_a)):
        result[row][0] = matrix_a[row][0] * matrix_b[row][0]
    return result

def matrix_add_begin_column(matrix, value):
    rows = len(matrix)
    cols = len(matrix[0])
    newcols = len(matrix[0]) + 1
    newmatrix = matrix_zeros(rows, newcols)
    
    for i in range(rows):
        newmatrix[i][0] = value
    
    for i in range(rows):
        for j in range(cols):
            newmatrix[i][j + 1] = matrix[i][j]
    
    return newmatrix

def matrix_add_end_column_potential_last(matrix, potential):
    rows = len(matrix)
    cols = len(matrix[0])
    newcols = len(matrix[0]) + 1
    newmatrix = matrix_zeros(rows, newcols)
    
    for i in range(rows):
        newmatrix[i][-1] = matrix[i][-1]**potential
    
    for i in range(rows):
        for j in range(cols):
            newmatrix[i][j] = matrix[i][j]
    
    return newmatrix

def matrix_check_squareness(A):
    if len(A) != len(A[0]):
        raise ArithmeticError("Matrix must be square to inverse.")

def matrix_determinant(A, total=0):
    indices = list(range(len(A)))
    
    if len(A) == 2 and len(A[0]) == 2:
        val = A[0][0] * A[1][1] - A[1][0] * A[0][1]
        return val

    for fc in indices:
        As = matrix_copy(A)
        As = As[1:]
        height = len(As)
        builder = 0

        for i in range(height):
            As[i] = As[i][0:fc] + As[i][fc+1:]

        sign = (-1) ** (fc % 2)
        sub_det = matrix_determinant(As)
        total += A[0][fc] * sign * sub_det

    return total

def matrix_check_non_singular(A):
    det = matrix_determinant(A)
    if det != 0:
        return det
    else:
        raise ArithmeticError("Singular Matrix!")

def matrix_check_equality(A,B, tol=None):
    if len(A) != len(B) or len(A[0]) != len(B[0]):
        return False

    for i in range(len(A)):
        for j in range(len(A[0])):
            if tol == None:
                if A[i][j] != B[i][j]:
                    return False
            else:
                if round(A[i][j],tol) != round(B[i][j],tol):
                    return False

    return True
        
def matrix_multiplication(matrix_a, matrix_b):
    rows_a = len(matrix_a)
    cols_a = len(matrix_a[0])
    rows_b = len(matrix_b)
    cols_b = len(matrix_b[0])

    if cols_a != rows_b:
      print("Incorrect dimensions.")
      return

    result = [[0 for row in range(cols_b)] for col in range(rows_a)] #creating result

    for i in range(rows_a):
        for j in range(cols_b):
            for k in range(cols_a):
                result[i][j] += matrix_a[i][k] * matrix_b[k][j]
    return result

def matrix_inverse(matrix_a, tol=None):
    matrix_check_squareness(matrix_a)
    matrix_check_non_singular(matrix_a)
    n = len(matrix_a)
    AM = matrix_copy(matrix_a)
    I = matrix_identity(n)
    IM = matrix_copy(I)
    indices = list(range(n))
    for fd in range(n):
        fdScaler = 1.0 / AM[fd][fd]
        for j in range(n):
            AM[fd][j] *= fdScaler
            IM[fd][j] *= fdScaler
        for i in indices[0:fd] + indices[fd+1:]: 
            crScaler = AM[i][fd]
            for j in range(n): 
                AM[i][j] = AM[i][j] - crScaler * AM[fd][j]
                IM[i][j] = IM[i][j] - crScaler * IM[fd][j]

    if matrix_check_equality(I, matrix_multiplication(matrix_a,IM), tol):
        return IM
    else:
        raise ArithmeticError("Matrix inverse out of tolerance.")

def matrix_identity(n):
    matrix_iden = matrix_zeros(n, n)
    for i in range(n):
        matrix_iden[i][i] = 1
    return matrix_iden

def matrix_transpose(matrix):
    rows = len(matrix)
    cols = len(matrix[0])

    matrixtrans = matrix_zeros(cols, rows)

    for i in range(rows):
        for j in range(cols):
            matrixtrans[j][i] = matrix[i][j]

    return matrixtrans
    
def linear_least_squares(matrix, y): # B = (X^T * X)^-1 * X^t * y
    newmatrix = matrix_add_begin_column(matrix, 1)
    matrixtranspose = matrix_transpose(newmatrix)
    section_1 = matrix_inverse(matrix_multiplication(matrixtranspose, newmatrix), 1) # (X^T * X)^-1
    section_2 = matrix_multiplication(section_1, matrixtranspose) # (X^T * X)^-1 * X^t
    section_3 = matrix_multiplication(section_2, y) # (X^T * X)^-1 * X^t * y
    
    return section_3

def quadratic_least_squares(matrix, y): # B = (X^T * X)^-1 * X^t * y
    newmatrix = matrix_add_end_column_potential_last(matrix_add_begin_column(matrix, 1), 2)
    matrixtranspose = matrix_transpose(newmatrix)
    section_1 = matrix_inverse(matrix_multiplication(matrixtranspose, newmatrix), 1) # (X^T * X)^-1
    section_2 = matrix_multiplication(section_1, matrixtranspose) # (X^T * X)^-1 * X^t
    section_3 = matrix_multiplication(section_2, y) # (X^T * X)^-1 * X^t * y

    return section_3

def robust_least_squares(matrix, y): # B = (X^T * W.X)^-1 * X^t * W.y
    matrixlinear = linear_least_squares(matrix, y)
    newy = linear_predict_matrix(matrix, matrixlinear)    
    w = matrix_copy(newy)
    
    # calculating w
    for i in range(len(w)):
        w[i][0] = 1/(abs(y[i][0] - newy[i][0]))    
    
    y = matrix_escalar_matrix_multiplication(y, w)
    matrix = matrix_escalar_matrix_multiplication(matrix, w)
    
    return linear_least_squares(matrix, y)

def linear_predict(matrix_prediction_test, b):     
    matrix = matrix_add_begin_column(matrix_prediction_test, 1)
    result = matrix_multiplication(matrix, b)
    return result[0][0]

def quadratic_predict(matrix_prediction_test, b):
    matrix = matrix_add_end_column_potential_last(matrix_add_begin_column(matrix_prediction_test, 1), 2)
    result = matrix_multiplication(matrix, b)
    return result[0][0]

def linear_predict_matrix(matrix, b):
    result = matrix_zeros(len(matrix), 1)
    for row in range(len(matrix)):
        result[row][0] = linear_predict([matrix[row]], b)
    return result
    
def dataset_test(path, prediction_array = None, datatype="float"):
    prediction_test = []
    if(prediction_array != None):
        prediction_test = matrix_array_to_matrix(prediction_array)
    
    print(">>>>>> " + path + " <<<<<<")
    readmatrix = matrix_read_csv(path, ";", datatype)
    matrix = numpy.delete(readmatrix, len(readmatrix[0]) - 1, 1) #extract features    
    # extract y
    y = matrix_copy(readmatrix)
    for col in range(len(readmatrix[0]) - 1):
        y = numpy.delete(y, 0, 1)        
    
    result_lin = linear_least_squares(matrix, y)
    matrix_print("linear_least_squares", result_lin)
    if(prediction_array != None):
        matrix_print(">> Prediction for: ", prediction_test)
        print("Is: " + str(linear_predict(prediction_test, result_lin)))
        
    result_quad = quadratic_least_squares(matrix, y)
    matrix_print("quadratic_least_squares", result_quad)
    if(prediction_array != None):
        matrix_print(">> Prediction for: ", prediction_test)
        print("Is: " + str(quadratic_predict(prediction_test, result_quad)))
        
    result_robust = robust_least_squares(matrix, y)
    matrix_print("robust_least_squares", result_robust)
    if(prediction_array != None):
        matrix_print(">> Prediction for: ", prediction_test)
        print("Is: " + str(linear_predict(prediction_test, result_robust)))
    
    print("\n")
    
def exercise_test():
    dataset_test("../dataset/alpswater.csv", [190])
    dataset_test("../dataset/books_attend_grade.csv", [0, 9])
    dataset_test("../dataset/us_census.csv", [2010.0])    

In [335]:
exercise_test()

>>>>>> ../dataset/alpswater.csv <<<<<<
linear_least_squares
[-81.0637]
[0.5229]
>> Prediction for: 
[190]
Is: 18.2858290204276
quadratic_least_squares
[38.8293]
[-0.6548]
[0.0029]
>> Prediction for: 
[190]
Is: 18.741470484161326
robust_least_squares
[-10.5061]
[0.1255]
>> Prediction for: 
[190]
Is: 13.337035125532354


>>>>>> ../dataset/books_attend_grade.csv <<<<<<
linear_least_squares
[37.3792]
[4.0369]
[1.2835]
>> Prediction for: 
[0, 9]
Is: 48.930480676960784
quadratic_least_squares
[64.0916]
[4.1689]
[-2.9102]
[0.1484]
>> Prediction for: 
[0, 9]
Is: 49.92158646276325
robust_least_squares
[1.1157]
[27.2588]
[0.0039]
>> Prediction for: 
[0, 9]
Is: 1.1508933978099845


>>>>>> ../dataset/us_census.csv <<<<<<
linear_least_squares
[-3783.9456]
[2.0253]
>> Prediction for: 
[2010.0]
Is: 286.9128909090773
quadratic_least_squares
[32294.0174]
[-34.9875]
[0.0095]
>> Prediction for: 
[2010.0]
Is: 311.5880717733089
robust_least_squares
[-16.8362]
[0.1122]
>> Prediction for: 
[2010.0]
Is: 208.7